# Cityscapes gtFine Reorganizer - Local Edition

This notebook reorganizes your existing Cityscapes gtFine dataset from a local folder:

**Current structure:**
```
gtFine/
├── train/berlin/, train/bonn/, etc.
├── val/berlin/, val/munich/, etc.
└── test/berlin/, test/bielefeld/, etc.
```

**Target structure:**
```
reorganized_cityscapes/
└── gtFine/
    ├── train/building/, train/non_building/
    ├── val/building/, val/non_building/
    └── test/building/, test/non_building/
```

This will reorganize only the gtFine annotation files (no RGB images needed).

## Step 1: Configure Local Paths and Locate Your Data

In [ ]:
# Configure local paths
import os

# Path configuration - update these to match your local directory structure
# Example: GTFINE_PATH = "/path/to/your/gtFine"
#          OUTPUT_PATH = "/path/to/your/reorganized_cityscapes"
GTFINE_PATH = "./gtFine"  # Update this to your gtFine directory path
OUTPUT_PATH = "./reorganized_cityscapes"  # Update this to your desired output directory

print("✅ Path configuration loaded!")
print(f"📁 gtFine path: {GTFINE_PATH}")
print(f"💾 Output path: {OUTPUT_PATH}")

# Verify your data exists
print("\n🔍 Checking your data...")
if os.path.exists(GTFINE_PATH):
    print(f"✅ Found gtFine directory")
    splits = [d for d in os.listdir(GTFINE_PATH) if os.path.isdir(os.path.join(GTFINE_PATH, d))]
    print(f"   Splits found: {splits}")
else:
    print(f"❌ gtFine directory not found at {GTFINE_PATH}")
    print(f"   Please update GTFINE_PATH in the cell above to point to your gtFine directory")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive mounted!
📁 gtFine path: /content/drive/MyDrive/gtFine
💾 Output path: /content/drive/MyDrive/reorganized_cityscapes

🔍 Checking your data...
✅ Found gtFine directory
   Splits found: ['train', 'val', 'test']


## Step 2: Preview Current Structure

In [ ]:
# Preview your current dataset structure
print("Current dataset structure:")
print("=" * 50)

for split in ['train', 'val', 'test']:
    gtfine_split_path = os.path.join(GTFINE_PATH, split)
    if os.path.exists(gtfine_split_path):
        cities = [d for d in os.listdir(gtfine_split_path)
                 if os.path.isdir(os.path.join(gtfine_split_path, d))]
        print(f"\n📁 gtFine/{split}/")
        print(f"   Cities: {cities[:5]}{'...' if len(cities) > 5 else ''} ({len(cities)} total)")

        # Count total images in this split
        total_images = 0
        for city in cities:
            city_path = os.path.join(gtfine_split_path, city)
            json_files = [f for f in os.listdir(city_path) if f.endswith('_polygons.json')]
            total_images += len(json_files)
        print(f"   Total images: {total_images}")

print("\n🎯 After reorganization:")
print("Each split will have only 2 directories:")
print("   ├── building/     (images containing buildings)")
print("   └── non_building/ (images without buildings)")

Current dataset structure:

📁 gtFine/train/
   Cities: ['aachen', 'cologne', 'bochum', 'darmstadt', 'bremen']... (18 total)
   Total images: 2975

📁 gtFine/val/
   Cities: ['lindau', 'frankfurt', 'munster'] (3 total)
   Total images: 500

📁 gtFine/test/
   Cities: ['berlin', 'bonn', 'leverkusen', 'mainz', 'bielefeld']... (6 total)
   Total images: 1525

🎯 After reorganization:
Each split will have only 2 directories:
   ├── building/     (images containing buildings)
   └── non_building/ (images without buildings)


## Step 3: Building Detection and Reorganization Functions

In [ ]:
import json
import glob
import shutil
from pathlib import Path

def has_building_annotation(annotation_file):
    """
    Check if an annotation file contains building objects.

    Args:
        annotation_file (str): Path to the JSON annotation file

    Returns:
        bool: True if the image contains buildings, False otherwise
    """
    try:
        with open(annotation_file, 'r') as f:
            data = json.load(f)

        # Check if any object in the annotation is a building
        objects = data.get('objects', [])
        for obj in objects:
            if obj.get('label') == 'building':
                return True
        return False
    except Exception as e:
        print(f"Warning: Could not process {annotation_file}: {e}")
        return False

def get_base_name_from_annotation(annotation_file):
    """Extract base name from annotation filename."""
    base_name = os.path.basename(annotation_file)
    if '_gtFine_polygons.json' in base_name:
        return base_name.replace('_gtFine_polygons.json', '')
    elif '_gtCoarse_polygons.json' in base_name:
        return base_name.replace('_gtCoarse_polygons.json', '')
    return base_name

def find_gtfine_files(base_name, gtfine_path, split):
    """
    Find all gtFine files related to a base image name.

    Args:
        base_name (str): Base name like "city_123456_123456"
        gtfine_path (str): Path to gtFine data
        split (str): Split name (train, val, test)

    Returns:
        list: List of gtFine file paths
    """
    files = []
    city = base_name.split('_')[0]

    # gtFine files
    gtfine_city_path = os.path.join(gtfine_path, split, city)
    if os.path.exists(gtfine_city_path):
        gtfine_patterns = [
            f"{base_name}_gtFine_color.png",
            f"{base_name}_gtFine_instanceIds.png",
            f"{base_name}_gtFine_labelIds.png",
            f"{base_name}_gtFine_polygons.json"
        ]
        for pattern in gtfine_patterns:
            file_path = os.path.join(gtfine_city_path, pattern)
            if os.path.exists(file_path):
                files.append(file_path)

    return files

def copy_gtfine_files(file_paths, output_path, split, category):
    """
    Copy gtFine files to the new building-based directory structure.

    Args:
        file_paths (list): List of gtFine file paths
        output_path (str): Output directory path
        split (str): Split name
        category (str): 'building' or 'non_building'
    """
    # Create new directory structure
    new_dir = os.path.join(output_path, 'gtFine', split, category)
    os.makedirs(new_dir, exist_ok=True)

    for file_path in file_paths:
        new_file_path = os.path.join(new_dir, os.path.basename(file_path))
        try:
            shutil.copy2(file_path, new_file_path)
        except Exception as e:
            print(f"Warning: Could not copy {file_path}: {e}")

print("✅ gtFine reorganization functions loaded!")

✅ gtFine reorganization functions loaded!


## Step 4: Run the Reorganization

In [ ]:
# Main reorganization function for gtFine only
def reorganize_gtfine_dataset(gtfine_path, output_path, splits=['train', 'val', 'test']):
    """
    Reorganize the Cityscapes gtFine dataset by building content.

    Args:
        gtfine_path (str): Path to gtFine data
        output_path (str): Output directory
        splits (list): List of splits to process
    """
    print(f"🚀 Starting gtFine dataset reorganization...")
    print(f"📂 gtFine source: {gtfine_path}")
    print(f"💾 Output: {output_path}")
    print(f"📋 Splits: {', '.join(splits)}")

    # Create output directory
    os.makedirs(output_path, exist_ok=True)

    building_count = 0
    non_building_count = 0
    total_processed = 0

    for split in splits:
        print(f"\n🔄 Processing split: {split}")

        # Find all annotation files in this split
        annotation_pattern = os.path.join(gtfine_path, split, '*', '*_gtFine_polygons.json')
        annotation_files = glob.glob(annotation_pattern)

        if not annotation_files:
            print(f"  ⚠️ No annotation files found for split {split}")
            continue

        annotation_files.sort()
        print(f"  📄 Found {len(annotation_files)} annotation files")

        for i, annotation_file in enumerate(annotation_files):
            total_processed += 1

            # Extract base name
            base_name = get_base_name_from_annotation(annotation_file)

            # Check if image has building annotations
            has_buildings = has_building_annotation(annotation_file)
            category = 'building' if has_buildings else 'non_building'

            if has_buildings:
                building_count += 1
            else:
                non_building_count += 1

            # Find all related gtFine files for this base name
            gtfine_files = find_gtfine_files(base_name, gtfine_path, split)

            if gtfine_files:
                copy_gtfine_files(gtfine_files, output_path, split, category)

            # Progress update
            if (i + 1) % 100 == 0:
                print(f"    ✅ Processed {i + 1}/{len(annotation_files)} files...")

    print(f"\n🎉 gtFine reorganization complete!")
    print(f"📊 Results:")
    print(f"   🏢 Building images: {building_count}")
    print(f"   🌆 Non-building images: {non_building_count}")
    print(f"   📝 Total processed: {total_processed}")
    print(f"   💾 Output saved to: {output_path}")

    return building_count, non_building_count

# Run the reorganization
print("Starting gtFine reorganization... This may take several minutes!")
building_count, non_building_count = reorganize_gtfine_dataset(
    gtfine_path=GTFINE_PATH,
    output_path=OUTPUT_PATH
)

Starting gtFine reorganization... This may take several minutes!
🚀 Starting gtFine dataset reorganization...
📂 gtFine source: /content/drive/MyDrive/gtFine
💾 Output: /content/drive/MyDrive/reorganized_cityscapes
📋 Splits: train, val, test

🔄 Processing split: train
  📄 Found 2975 annotation files
    ✅ Processed 100/2975 files...
    ✅ Processed 200/2975 files...
    ✅ Processed 300/2975 files...
    ✅ Processed 400/2975 files...
    ✅ Processed 500/2975 files...
    ✅ Processed 600/2975 files...
    ✅ Processed 700/2975 files...
    ✅ Processed 800/2975 files...
    ✅ Processed 900/2975 files...
    ✅ Processed 1000/2975 files...
    ✅ Processed 1100/2975 files...
    ✅ Processed 1200/2975 files...
    ✅ Processed 1300/2975 files...
    ✅ Processed 1400/2975 files...
    ✅ Processed 1500/2975 files...
    ✅ Processed 1600/2975 files...
    ✅ Processed 1700/2975 files...
    ✅ Processed 1800/2975 files...
    ✅ Processed 1900/2975 files...
    ✅ Processed 2000/2975 files...
    ✅ Proce

## Step 5: Verify the New Structure

In [ ]:
# Verify the new gtFine structure
print("🔍 Verifying new gtFine dataset structure:")
print("=" * 60)

# Check gtFine structure
gtfine_path = os.path.join(OUTPUT_PATH, 'gtFine')
if os.path.exists(gtfine_path):
    print(f"\n📁 gtFine/")

    for split in ['train', 'val', 'test']:
        split_path = os.path.join(gtfine_path, split)
        if os.path.exists(split_path):
            print(f"  📂 {split}/")

            # Count files in building directory
            building_path = os.path.join(split_path, 'building')
            if os.path.exists(building_path):
                building_files = len([f for f in os.listdir(building_path)])
                print(f"    🏢 building/: {building_files} files")

            # Count files in non_building directory
            non_building_path = os.path.join(split_path, 'non_building')
            if os.path.exists(non_building_path):
                non_building_files = len([f for f in os.listdir(non_building_path)])
                print(f"    🌆 non_building/: {non_building_files} files")
        else:
            print(f"  📂 {split}/: Not found")
else:
    print(f"\n📁 gtFine/: Not found")

# Show sample files
print(f"\n📄 Sample files in reorganized structure:")
sample_building_path = os.path.join(OUTPUT_PATH, 'gtFine', 'train', 'building')
if os.path.exists(sample_building_path):
    sample_files = os.listdir(sample_building_path)[:5]
    for file in sample_files:
        print(f"  🏢 {file}")

sample_non_building_path = os.path.join(OUTPUT_PATH, 'gtFine', 'train', 'non_building')
if os.path.exists(sample_non_building_path):
    sample_files = os.listdir(sample_non_building_path)[:5]
    for file in sample_files:
        print(f"  🌆 {file}")

print(f"\n✅ gtFine reorganization verification complete!")
print(f"📁 Your new gtFine dataset is available at: {OUTPUT_PATH}/gtFine")

## Step 6: Summary and Next Steps

🎉 **Congratulations!** Your Cityscapes gtFine dataset has been successfully reorganized!

### What was accomplished:
- ✅ Analyzed all gtFine annotation files to detect building content
- ✅ Reorganized from city-based to building-based directory structure  
- ✅ Preserved all gtFine file relationships (annotations, labels, instances)
- ✅ Created simple 2-directory structure under each split

### Your new gtFine dataset structure:
```
reorganized_cityscapes/
└── gtFine/
    ├── train/building/ & train/non_building/
    ├── val/building/ & val/non_building/
    └── test/building/ & test/non_building/
```

### File types included:
- `*_gtFine_color.png` - Colored label images
- `*_gtFine_instanceIds.png` - Instance segmentation masks
- `*_gtFine_labelIds.png` - Label ID images
- `*_gtFine_polygons.json` - Polygon annotations

### Next steps:
1. **Use for training**: Load `building/` directories for building-focused models
2. **Annotation analysis**: Analyze building vs non-building annotation patterns
3. **Data loading**: Simplified data loading with only 2 categories per split
4. **Model development**: Train specialized models on building vs non-building annotations

Your reorganized gtFine dataset is now saved locally and ready for use! 🚀